In [2]:
#Importing Packages

import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.optimize import least_squares
from scipy.optimize import differential_evolution
from scipy.optimize import minimize
from scipy.interpolate import make_interp_spline
import sympy as smp
from sympy.core.function import AppliedUndef
import time
import random as rd
from mpmath import mp
mp.dps = 32

import ipywidgets as widgets
from IPython.display import display

from sympy.polys.polyfuncs import interpolate
import importlib
from IPython.display import display, Math

smp.init_printing(use_latex='mathjax') 
ln = np.log

import GravastarClass
GravastarClass = importlib.reload(GravastarClass)
from GravastarClass import *

approach = "fh"
if approach == "fb":
    import fbBoundaryLayer as fds
    fds = importlib.reload(fds)
    from fbBoundaryLayer import *
elif approach == "fh":
    import fhBoundaryLayer as fds
    fds = importlib.reload(fds)
    from fhBoundaryLayer import *
elif approach == "fb_EH":
    import fbEinsteinHilbert as fds
    fds = importlib.reload(fds)
    from fbEinsteinHilbert import *
elif approach == "fh_exp":
    import fhExponential as fds
    fds = importlib.reload(fds)
    from fhExponential import *

runtime = "uncalculated"

In [3]:
def add_new_points(action, new_points = 1, double = True):
    global N
    N = action.fields.N
    global S
    if double:
        N += new_points*(N-3)
    else:
        N += new_points
    ansatz_list = []
    for fld in action.field_list:
        if fld.dynamic:
            ansatz_list.append(interp1d(fld.x_list, fld.v, kind = "cubic"))
        else:
            ansatz_list.append(fld.ansatz)

    fds.create_fields(ansatz_list, [-100, 100, N])
    S = Action(fds.Ltot, fds.y, lambdify_type = action.lambdify_type, params = action.params)

In [4]:
#Set Problem Parameters

N = 50 #Total number of points

gamma = 1
alpha = 1
beta = 1
epsilon = 1e-4

StartingParameters = {alpha_s : alpha, beta_s : beta, gamma_s : gamma}

#maximum and minimum integral bounds
xm = -100
xp = 100

initialize_fds(xm, xp, N, alpha, beta, gamma, epsilon, T = 30)

In [8]:
boundary = 3 * fds.w.s[1]
S = Action(fds.Ltot, fds.y, lambdify_type = "numpy", params = StartingParameters, boundary_term = boundary, lambdify_deriv = False, calcderiv = True)

Initializing Action
Calculating First Derivatives
Calculating First Derivatives for f
Calculating First Derivatives for h
Calculating First Derivatives for varphi
Calculating First Derivatives for psi
Calculating First Derivatives for w

Calculating Second Derivatives
Calculating Second Derivatives for f
Calculating Second Derivatives for h
Calculating Second Derivatives for varphi
Calculating Second Derivatives for psi
Calculating Second Derivatives for w


In [9]:
dynamic_list_q = []
for smb in fds.ph.s + fds.psi.s + fds.w.s:
    if isinstance(smb, smp.Symbol):
        dynamic_list_q.append(smb)

dynamic_list_f = []
for smb in fds.f.s:
    if isinstance(smb, smp.Symbol):
        dynamic_list_f.append(smb)

In [10]:
fds.y.reset_values()
S.runNR(100, dynamic = dynamic_list_q, PureNR = False, lambdify_first = True, revert_if_failed = False)
plot_all_fields()
S.runNR(5, dynamic = dynamic_list_f, PureNR = True, lambdify_first = True, revert_if_failed = False)
plot_all_fields()

Newton Raphson Converged to Square Value of 5.22e-22 after 1 steps


Newton Raphson Only Converged to Square Value of 3.76e+01
